In [1]:
### This is code for P300 signal classification for across-subject paradigm.

# Steps involved
# 1. Load the training data
# 2. Prepare data for training
# 3. Train the models
# 4. Test with across-subjects data 

## 1. Load the training data. BCI2000 package is used to load data. 

import numpy as np
import pandas as pd

from scipy import stats
import os
from BCI2kReader import BCI2kReader as b2k 

src_dir = '../CV-test/
#out_dir = 'Desktop/eeg-data/output/'+ subject_name +'_results.pdf'
list_dir = os.listdir(src_dir)
print (list_dir)
subject_to_signals =  {}  #dictionary; key= subject, value = data

for sub_root_dir in os.listdir(src_dir):

    if not sub_root_dir.startswith("."):
        
        for subject_dir in os.listdir(src_dir+sub_root_dir+'/')[0:85]:
   
            if not subject_dir.startswith("."):
                    if (subject_dir.endswith(".dat")):
                        with b2k.BCI2kReader(src_dir+sub_root_dir+'/'+subject_dir, "BaggingTrain.dat") as file: 
                            subject_to_signals[subject_dir] = file.read()


SyntaxError: EOL while scanning string literal (3686175488.py, line 18)

In [2]:
# Step 2: Prepare the data for training
# Note that we need to format the data for training EEGNET, RG+XDAWN, EEG_INCEPTION and flattened data for rLDA training 

window = 192   #Signal time series window
sht_window = 128 # shoretened window for LDA analysis
subject_name = []
z_score = []
#zerozscore = []
#print (subject_to_signals)
array_elements = sht_window * 32

Train_data1 = np.array(np.empty((0,32,window), float))  # Training data array initialization
Train_Labels1 = []   # Training label data initialization

Train_Labels_Flattened1 = []   # Training label data initialization
Train_data_flattened1 = np.array(np.empty((0,array_elements), float))  # Training data array initialization


for subject, my_signals in subject_to_signals.items():
   # print (my_signals)
    Train_data = np.array(np.empty((0,32,window), float))  # Training data array initialization
    Train_Labels = []   # Training label data initialization
    Train_Labels_Flattened = []   # Training label data initialization
    Train_data_flattened = np.array(np.empty((0,array_elements), float))  # Training data array initialization

    stimuluscode = (my_signals[1]['StimulusCode']) #ndarrays ; target 
    stimulustype = (my_signals[1]['StimulusType']) #p3label
    data = stimuluscode[0] #target numbers
    data_label = stimulustype[0] #p3label numbers
    rownum = len(data)
    labelrow = len(data_label)
    zerozscore = (-1*np.mean(my_signals[0])/(my_signals[0]).std())
    
    change = [] #indexes of 0 where next index is nonzero for target
    for x in range(rownum-1):    
        if bool(data.item(x) == 0) and bool(data.item(x + 1) != 0): 
                change.append(x)
      
    
    
    attended = [] #indexes where p3label value is 1
    nonattended = [] #indexes where p3label value is 0
    
    #Train_data = np.array(np.empty((0,32,window), float))  # Training data array initialization
    #Train_Labels = []   # Training label data initialization
    #Train_Labels_Flattened = []   # Training label data initialization
    #Train_data_flattened = np.array(np.empty((0,array_elements), float))  # Training data array initialization


    nonattended_count =1
   
    for x in range(len(change)): 
        attended_values = np.empty((0,window), float) #2D array with train data each row in same a time frame
        nonattended_values = np.empty((0,window), float)
        attended_values_sht = np.empty((0,sht_window), float) #2D array with train data each row in same a time frame
        nonattended_values_sht = np.empty((0,sht_window), float)
 
        if bool(data_label.item(change[x] + 1) == 0): 
            nonattended_count =nonattended_count +1    
            nonattended.append(change[x])
            Train_Labels.append(int(0))
            for num in range(len(my_signals[0])):
                electrode = my_signals[0][num]
                zelectrode = stats.zscore(my_signals[0][num])
                nonattendedtemp = []
                nonattendedtemp_sht =[]
                if (change[x] + window) > len(electrode):
                    addtime = len(electrode) - (change[x])            
                    for y in range(addtime):
                        nonattendedtemp.append(zelectrode[change[x] + y])
                    for y in range(sht_window):
                        nonattendedtemp_sht.append(zelectrode[change[x] + y])
                    nonattended_values_sht = np.append(nonattended_values_sht, np.array([nonattendedtemp_sht]), axis = 0)               
                    nonattended_values = np.append(nonattended_values, np.array([nonattendedtemp]), axis = 0)
                else:
                    for y in range(window):
                        nonattendedtemp.append(zelectrode[change[x] + y])
                    for y in range(sht_window):
                        nonattendedtemp_sht.append(electrode[change[x] + y])
                nonattended_values = np.append(nonattended_values, np.array([nonattendedtemp]), axis = 0)
                nonattended_values_sht = np.append(nonattended_values_sht, np.array([nonattendedtemp_sht]), axis = 0)               
            #if nonattended_count % 5 == 0:
            Train_data_flattened = np.append(Train_data_flattened, np.array([nonattended_values_sht.flatten()]), axis=0)
            Train_Labels_Flattened.append(int(0))
            Train_data = np.append(Train_data,np.array([nonattended_values]), axis=0)
        else: 
            attended.append(change[x] + 1) 
            Train_Labels.append(int(1))
            Train_Labels_Flattened.append(int(1))
          
            for num in range(len(my_signals[0])):
                electrode = my_signals[0][num]
                zelectrode = stats.zscore(my_signals[0][num])
                attendedtemp = []
                attendedtemp_sht = []
        
                for y in range(window): 
                    attendedtemp.append(zelectrode[change[x] + y])
                for y in range(sht_window): 
                    attendedtemp_sht.append(electrode[change[x] + y])

                attended_values = np.append(attended_values, np.array([attendedtemp]), axis = 0)
                attended_values_sht = np.append(attended_values_sht, np.array([attendedtemp_sht]), axis = 0)

            Train_data_flattened = np.append( Train_data_flattened, np.array([attended_values_sht.flatten()]), axis=0)
            Train_data = np.append( Train_data,np.array([attended_values]), axis=0)        
    
    Train_data_flattened1 = np.concatenate((Train_data_flattened1, Train_data_flattened),axis=0) # format is in (trials, channels, samples)
    Train_Labels_Flattened1 = np.concatenate((Train_Labels_Flattened1, Train_Labels_Flattened),axis=0) 
    Train_data1 = np.concatenate((Train_data1, Train_data),axis=0) # format isTrain_Labels_Flattened1 in (trials, channels, samples)
    Train_Labels1 = np.concatenate((Train_Labels1, Train_Labels),axis=0) 
    print("no of labled P300 events:"+str(len(attended)))
    print("no of non_attended P300 events:"+str(len(nonattended)))
    
print(np.shape(Train_data))
print(np.shape(Train_Labels))
print(np.shape(Train_data1))
print(np.shape(Train_Labels1))

print(np.shape(Train_Labels_Flattened1))
print(np.shape(Train_data_flattened1))

no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:

In [4]:
#3.1 EEGNet Model Training

# EEGNet copied code for model fit and testing

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold,GridSearchCV



# tools for plotting confusion matrices
from matplotlib import pyplot as plt

# while the default tensorflow ordering is 'channels_last' we set it here
# to be explicit in case if the user has changed the default ordering
K.set_image_data_format('channels_last')

# extract raw data. scale by 1000 due to scaling sensitivity in deep learning
X = Train_data1 # format is in (trials, channels, samples)
y = Train_Labels1
kernels, chans, samples = 1, 32, 192

#split into training and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=42)


############################# EEGNet portion ##################################

# convert labels to one-hot encodings.
Y_train      = np_utils.to_categorical(Y_train,  num_classes=2)
#Y_validate   = np_utils.to_categorical(Y_validate,  num_classes=2)
Y_test       = np_utils.to_categorical(Y_test,  num_classes=2)

# convert data to NHWC (trials, channels, samples, kernels) format. Data 
# contains 60 channels and 151 time-points. Set the number of kernels to 1.
X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
#X_validate   = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
   

# configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = 0.5, kernLength = 96, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

# compile the model and set the optimizers
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

###############################################################################
# if the classification task was imbalanced (significantly more trials in one
# class versus the others) you can assign a weight to each class during 
# optimization to balance it out. This data is approximately balanced so we 
# don't need to do this, but is shown here for illustration/completeness. 
###############################################################################

# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1
class_weights = {0:5, 1:1}

################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
# Riemannian geometry classification (below)
################################################################################
fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300, 
                        verbose = 2,  validation_split=0.2,
                        #validation_data=(X_validate, Y_validate),
                        callbacks=[checkpointer], class_weight = class_weights)

# load optimal weights
model.load_weights('/tmp/checkpoint.h5')

###############################################################################
# can alternatively used the weights provided in the repo. If so it should get
# you 93% accuracy. Change the WEIGHTS_PATH variable to wherever it is on your
# system.
###############################################################################

# WEIGHTS_PATH = /path/to/EEGNet-8-2-weights.h7 
# model.load_weights(WEIGHTS_PATH)

###############################################################################
# make prediction on test set.
###############################################################################

probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == Y_test.argmax(axis=-1))
print("EEGNet-8,2 Classification accuracy: %f " % (acc))


names        = ['Non-Attended', 'Attended']


from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    
def display_results(y_true, y_preds, class_labels):
    
    results = pd.DataFrame(precision_recall_fscore_support(y_true, y_preds),
                          columns=class_labels).T

    results.rename(columns={0: 'Precision', 1: 'Recall',
                            2: 'F-Score', 3: 'Support'}, inplace=True)
    
    results.sort_values(by='F-Score', ascending=False, inplace=True)                           
    global_acc = accuracy_score(y_true, y_preds)
    
    print("Overall Categorical Accuracy: {:.2f}%".format(global_acc*100))
    return results
display_results(Y_test.argmax(axis = -1), preds, names)
#display_results(Y_test.argmax(axis=-1), preds, names)



2023-03-15 07:26:37.196529: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/300

Epoch 00001: val_loss improved from inf to 0.36939, saving model to /tmp/checkpoint.h5
4464/4464 - 276s - loss: 0.7187 - accuracy: 0.8343 - val_loss: 0.3694 - val_accuracy: 0.8411 - 276s/epoch - 62ms/step
Epoch 2/300

Epoch 00002: val_loss improved from 0.36939 to 0.29797, saving model to /tmp/checkpoint.h5
4464/4464 - 267s - loss: 0.4809 - accuracy: 0.8616 - val_loss: 0.2980 - val_accuracy: 0.8843 - 267s/epoch - 60ms/step
Epoch 3/300

Epoch 00003: val_loss improved from 0.29797 to 0.27711, saving model to /tmp/checkpoint.h5
4464/4464 - 245s - loss: 0.4444 - accuracy: 0.8756 - val_loss: 0.2771 - val_accuracy: 0.8928 - 245s/epoch - 55ms/step
Epoch 4/300

Epoch 00004: val_loss improved from 0.27711 to 0.27102, saving model to /tmp/checkpoint.h5
4464/4464 - 242s - loss: 0.4348 - accuracy: 0.8792 - val_loss: 0.2710 - val_accuracy: 0.8977 - 242s/epoch - 54ms/step
Epoch 5/300

Epoch 00005: val_loss did not improve from 0.27102
4464/4464 - 242s - loss: 0.4269 - accuracy: 0.8829 -

,Precision,Recall,F-Score,Support
Non-Attended,0.914837,0.994516,0.953014,18600.0
Attended,0.951429,0.537097,0.686598,3720.0


In [6]:
# 3.2 Code for trainig the RG+xDAWN algorithm


# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt

# tools for plotting confusion matrices
from matplotlib import pyplot as plt

# while the default tensorflow ordering is 'channels_last' we set it here
# to be explicit in case if the user has changed the default ordering
K.set_image_data_format('channels_last')

# extract raw data. scale by 1000 due to scaling sensitivity in deep learning
X = Train_data1 # format is in (trials, channels, samples)
y = Train_Labels1
kernels, chans, samples = 1, 32, 192



X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=42)


############################# EEGNet portion ##################################

# convert labels to one-hot encodings.
Y_train      = np_utils.to_categorical(Y_train,  num_classes=2)
#Y_validate   = np_utils.to_categorical(Y_validate,  num_classes=2)
Y_test       = np_utils.to_categorical(Y_test,  num_classes=2)

# convert data to NHWC (trials, channels, samples, kernels) format. Data 
# contains 60 channels and 151 time-points. Set the number of kernels to 1.
X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
#X_validate   = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)

n_components = 2  # pick some components

# set up sklearn pipeline
clf = make_pipeline(XdawnCovariances(n_components),
                    TangentSpace(metric='riemann'),
                    LogisticRegression())

preds_rg     = np.zeros(len(Y_test))

# reshape back to (trials, channels, samples)
X_train      = X_train.reshape(X_train.shape[0], chans, samples)
X_test       = X_test.reshape(X_test.shape[0], chans, samples)

# train a classifier with xDAWN spatial filtering + Riemannian Geometry (RG)
# labels need to be back in single-column format
clf.fit(X_train, Y_train.argmax(axis = -1))
preds_rg     = clf.predict(X_test)

# Printing the results
acc2         = np.mean(preds_rg == Y_test.argmax(axis = -1))
print("xDAWN + RG Classification accuracy: %f " % (acc2))


   

xDAWN + RG Classification accuracy: 0.911873 


In [7]:
# 3.3 Code for training the rLDA algorithm

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold,GridSearchCV


X1 = Train_data_flattened1 # format is in (trials, channels, samples)
y1 = Train_Labels_Flattened1
kernels, chans, samples = 1, 32, 128
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1, y1, test_size=0.20, random_state=42)
lda_model = LinearDiscriminantAnalysis(shrinkage='auto', solver='eigen')
    #class_weights = {0:0.83, 1:0.17}

#class_weights = {0:0.83, 1:0.17}
lda_model.fit_transform(X1_train, Y1_train)
pred_LDA = lda_model.predict(X1_test)


acc3         = np.mean(pred_LDA == Y1_test)
print("LDA Classification accuracy: %f " % (acc3))
names        = ['Non-Attended', 'Attended']


LDA Classification accuracy: 0.893772 


In [8]:
# 4. Code for testing the models and outputing results to a CSV files 

import csv
from scipy import stats
import os
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from matplotlib import pyplot as plt


f=open('../eegnet-cv.csv', 'w')
csv_writer = csv.writer(f)

def display_results(y_true, y_preds, class_labels,method, subject):
    
    results = pd.DataFrame(precision_recall_fscore_support(y_true, y_preds),
                          columns=class_labels).T

    results.rename(columns={0: 'Precision', 1: 'Recall',
                            2: 'F-Score', 3: 'Support'}, inplace=True)
    
    results.sort_values(by='F-Score', ascending=False, inplace=True)                           
    global_acc = accuracy_score(y_true, y_preds)
    
    print(method + " Overall Categorical Accuracy: {:.2f}%".format(global_acc*100))
    print(results)
    data1=[subject,global_acc,results.iat[1,0],results.iat[1,1],results.iat[1,2], results.iat[1,3]]
    csv_writer.writerow(data1)
    return results

def get_data(subject_to_signals) :
    
    window = 192   #Signal time series window
    sht_window = 128 # shoretened window for LDA analysis
    subject_name = []
    z_score = []
    #zerozscore = []
    #print (subject_to_signals)
    array_elements = sht_window * 32

    Train_data1 = np.array(np.empty((0,32,window), float))  # Training data array initialization
    Train_Labels1 = []   # Training label data initialization

    Train_Labels_Flattened1 = []   # Training label data initialization
    Train_data_flattened1 = np.array(np.empty((0,array_elements), float))  # Training data array initialization



    for subject, my_signals in subject_to_signals.items():
       # print (my_signals)
        Train_data = np.array(np.empty((0,32,window), float))  # Training data array initialization
        Train_Labels = []   # Training label data initialization
        Train_Labels_Flattened = []   # Training label data initialization
        Train_data_flattened = np.array(np.empty((0,array_elements), float))  # Training data array initialization

        stimuluscode = (my_signals[1]['StimulusCode']) #ndarrays ; target 
        stimulustype = (my_signals[1]['StimulusType']) #p3label
        data = stimuluscode[0] #target numbers
        data_label = stimulustype[0] #p3label numbers
        rownum = len(data)
        labelrow = len(data_label)
        zerozscore = (-1*np.mean(my_signals[0])/(my_signals[0]).std())

        change = [] #indexes of 0 where next index is nonzero for target
        for x in range(rownum-1):    
            if bool(data.item(x) == 0) and bool(data.item(x + 1) != 0): 
                    change.append(x)



        attended = [] #indexes where p3label value is 1
        nonattended = [] #indexes where p3label value is 0

        #Train_data = np.array(np.empty((0,32,window), float))  # Training data array initialization
        #Train_Labels = []   # Training label data initialization
        #Train_Labels_Flattened = []   # Training label data initialization
        #Train_data_flattened = np.array(np.empty((0,array_elements), float))  # Training data array initialization


        nonattended_count =1

        for x in range(len(change)): 
            attended_values = np.empty((0,window), float) #2D array with train data each row in same a time frame
            nonattended_values = np.empty((0,window), float)
            attended_values_sht = np.empty((0,sht_window), float) #2D array with train data each row in same a time frame
            nonattended_values_sht = np.empty((0,sht_window), float)

            if bool(data_label.item(change[x] + 1) == 0): 
                nonattended_count =nonattended_count +1    
                nonattended.append(change[x])
                Train_Labels.append(int(0))
                for num in range(len(my_signals[0])):
                    electrode = my_signals[0][num]
                    zelectrode = stats.zscore(my_signals[0][num])
                    nonattendedtemp = []
                    nonattendedtemp_sht =[]
                    if (change[x] + window) > len(electrode):
                        addtime = len(electrode) - (change[x])            
                        for y in range(addtime):
                            nonattendedtemp.append(zelectrode[change[x] + y])
                        for y in range(sht_window):
                            nonattendedtemp_sht.append(zelectrode[change[x] + y])
                        nonattended_values_sht = np.append(nonattended_values_sht, np.array([nonattendedtemp_sht]), axis = 0)               
                        nonattended_values = np.append(nonattended_values, np.array([nonattendedtemp]), axis = 0)
                    else:
                        for y in range(window):
                            nonattendedtemp.append(zelectrode[change[x] + y])
                        for y in range(sht_window):
                            nonattendedtemp_sht.append(electrode[change[x] + y])
                    nonattended_values = np.append(nonattended_values, np.array([nonattendedtemp]), axis = 0)
                    nonattended_values_sht = np.append(nonattended_values_sht, np.array([nonattendedtemp_sht]), axis = 0)               
                #if nonattended_count % 5 == 0:
                Train_data_flattened = np.append(Train_data_flattened, np.array([nonattended_values_sht.flatten()]), axis=0)
                Train_Labels_Flattened.append(int(0))
                Train_data = np.append(Train_data,np.array([nonattended_values]), axis=0)
            else: 
                attended.append(change[x] + 1) 
                Train_Labels.append(int(1))
                Train_Labels_Flattened.append(int(1))

                for num in range(len(my_signals[0])):
                    electrode = my_signals[0][num]
                    zelectrode = stats.zscore(my_signals[0][num])
                    attendedtemp = []
                    attendedtemp_sht = []

                    for y in range(window): 
                        attendedtemp.append(zelectrode[change[x] + y])
                    for y in range(sht_window): 
                        attendedtemp_sht.append(electrode[change[x] + y])

                    attended_values = np.append(attended_values, np.array([attendedtemp]), axis = 0)
                    attended_values_sht = np.append(attended_values_sht, np.array([attendedtemp_sht]), axis = 0)

                Train_data_flattened = np.append( Train_data_flattened, np.array([attended_values_sht.flatten()]), axis=0)
                Train_data = np.append( Train_data,np.array([attended_values]), axis=0)        

        Train_data_flattened1 = np.concatenate((Train_data_flattened1, Train_data_flattened),axis=0) # format is in (trials, channels, samples)
        Train_Labels_Flattened1 = np.concatenate((Train_Labels_Flattened1, Train_Labels_Flattened),axis=0) 
        Train_data1 = np.concatenate((Train_data1, Train_data),axis=0) # format isTrain_Labels_Flattened1 in (trials, channels, samples)
        Train_Labels1 = np.concatenate((Train_Labels1, Train_Labels),axis=0) 
        print("no of labled P300 events:"+str(len(attended)))
        print("no of non_attended P300 events:"+str(len(nonattended)))

    print(np.shape(Train_data))
    print(np.shape(Train_Labels))
    print(np.shape(Train_data1))
    print(np.shape(Train_Labels1))
    print(np.shape(Train_data_flattened1))
    print(np.shape(Train_Labels_Flattened1))

    return Train_data1, Train_Labels1,Train_data_flattened1, Train_Labels_Flattened1

    #print(np.shape(Train_Labels_Flattened))
    #print(np.shape(Train_data_flattened))

    
#subject_name = 'subject_048'
src_dir = '../CV-test/' 
#out_dir = 'Desktop/eeg-data/output/'+ subject_name +'_results.pdf'
list_dir1 = os.listdir(src_dir)






names        = ['Non-Attended', 'Attended']
for sub_root_dir in os.listdir(src_dir):
    print(src_dir+sub_root_dir)
    if not (sub_root_dir.startswith(".") or sub_root_dir in list_dir):
        subject_to_signals =  {}  #dictionary; key= subject, value = data
        for subject_dir in os.listdir(src_dir+sub_root_dir+'/')[0:85]:
   
            if not subject_dir.startswith("."):
                    if (subject_dir.endswith(".dat")):
                        with b2k.BCI2kReader(src_dir+sub_root_dir+'/'+subject_dir, "EEGNet-Bagging-1.dat") as file: 
                            subject_to_signals[subject_dir] = file.read()
        X_test, Y_test, X1_test, Y1_test = get_data(subject_to_signals)
        X_test       = X_test.reshape(X_test.shape[0], chans, 192)
        Y_test       = np_utils.to_categorical(Y_test,  num_classes=2)    
        preds     = np.zeros(len(Y_test))
        probs       = model.predict(X_test)
        acc         = np.mean(probs == Y_test.argmax(axis=-1))
        print("EEGNT Classification accuracy: %f " % (acc))
        display_results(Y_test.argmax(axis = -1), probs, names , "EEGNET",sub_root_dir)

f.close()



f=open('../rg-cv.csv', 'w')
csv_writer = csv.writer(f)

for sub_root_dir in os.listdir(src_dir):
    print(src_dir+sub_root_dir)
    if not (sub_root_dir.startswith(".") or sub_root_dir in list_dir):
        subject_to_signals =  {}  #dictionary; key= subject, value = data
        for subject_dir in os.listdir(src_dir+sub_root_dir+'/')[0:85]:
   
            if not subject_dir.startswith("."):
                    if (subject_dir.endswith(".dat")):
                        with b2k.BCI2kReader(src_dir+sub_root_dir+'/'+subject_dir, "EEGNet-Bagging-2.dat") as file: 
                            subject_to_signals[subject_dir] = file.read()
        X_test, Y_test, X1_test, Y1_test = get_data(subject_to_signals)
        X_test       = X_test.reshape(X_test.shape[0], chans, 192)
        Y_test       = np_utils.to_categorical(Y_test,  num_classes=2)    
        preds     = np.zeros(len(Y_test))
        probs       = clf.predict(X_test)
        acc         = np.mean(probs == Y_test.argmax(axis=-1))
        print("RG+xDawn Classification accuracy: %f " % (acc))
        display_results(Y_test.argmax(axis = -1), probs, names , "RG+xDawn",sub_root_dir)

f.close()



f=open('../rLDA-cv.csv', 'w')
csv_writer = csv.writer(f)
for sub_root_dir in os.listdir(src_dir):
    print(src_dir+sub_root_dir)
    if not (sub_root_dir.startswith(".") or sub_root_dir in list_dir):
        subject_to_signals =  {}  #dictionary; key= subject, value = data
        for subject_dir in os.listdir(src_dir+sub_root_dir+'/')[0:85]:
   
            if not subject_dir.startswith("."):
                    if (subject_dir.endswith(".dat")):
                        with b2k.BCI2kReader(src_dir+sub_root_dir+'/'+subject_dir, "rLDA-Bagging-3.dat") as file: 
                            subject_to_signals[subject_dir] = file.read()
        X_test, Y_test, X1_test, Y1_test = get_data(subject_to_signals)
        pred_LDA = lda_model.predict(X1_test)
        acc         = np.mean(pred_LDA == Y1_test)
        print("rLDA Classification accuracy: %f " % (acc))
        display_results(Y1_test, pred_LDA, names , "rLDA",sub_root_dir)

f.close()





/users/raviravipati/Desktop/eeg-data/CV-test/subject_039
/users/raviravipati/Desktop/eeg-data/CV-test/subject_037
/users/raviravipati/Desktop/eeg-data/CV-test/.DS_Store
/users/raviravipati/Desktop/eeg-data/CV-test/subject_030
/users/raviravipati/Desktop/eeg-data/CV-test/subject_064


no of labled P300 events:220
no of non_attended P300 events:1100
no of labled P300 events:220
no of non_attended P300 events:1100
(1320, 32, 192)
(1320,)
(2640, 32, 192)
(2640,)
(2640, 4096)
(2640,)
(2640, 32, 192)
(2640, 2)
[0. 0. 0. ... 0. 0. 0.]
[1 0 0 ... 1 0 0]
RG+xDawn Classification accuracy: 0.887879 
RG+xDawn Overall Categorical Accuracy: 88.79%


              Precision    Recall   F-Score  Support
Non-Attended   0.897329  0.977273  0.935596   2200.0
Attended       0.795082  0.440909  0.567251    440.0
/users/raviravipati/Desktop/eeg-data/CV-test/subject_063


no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
(1200, 32, 192)
(1200,)
(2400, 32, 192)
(2400,)
(2400, 4096)
(2400,)
(2400, 32, 192)
(2400, 2)


[0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0]
RG+xDawn Classification accuracy: 0.934583 
RG+xDawn Overall Categorical Accuracy: 93.46%
              Precision  Recall   F-Score  Support
Non-Attended   0.956414  0.9655  0.960936   2000.0
Attended       0.818898  0.7800  0.798976    400.0
/users/raviravipati/Desktop/eeg-data/CV-test/subject_031
/users/raviravipati/Desktop/eeg-data/CV-test/subject_036
/users/raviravipati/Desktop/eeg-data/CV-test/subject_038
/users/raviravipati/Desktop/eeg-data/CV-test/subject_053


...

no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
(1200, 32, 192)
(1200,)
(2400, 32, 192)
(2400,)
(2400, 4096)
(2400,)
(2400, 32, 192)
(2400, 2)


[0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 1 1 0]
RG+xDawn Classification accuracy: 0.830833 
RG+xDawn Overall Categorical Accuracy: 83.08%
              Precision  Recall   F-Score  Support
Non-Attended   0.861287   0.950  0.903471   2000.0
Attended       0.484536   0.235  0.316498    400.0
/users/raviravipati/Desktop/eeg-data/CV-test/subject_045


no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
(1200, 32, 192)
(1200,)
(3600, 32, 192)
(3600,)
(3600, 4096)
(3600,)
(3600, 32, 192)
(3600, 2)


[0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 1 0 0]
RG+xDawn Classification accuracy: 0.942778 
RG+xDawn Overall Categorical Accuracy: 94.28%
              Precision    Recall   F-Score  Support
Non-Attended   0.939861  0.995000  0.966645   3000.0
Attended       0.964623  0.681667  0.798828    600.0
/users/raviravipati/Desktop/eeg-data/CV-test/subject_042


no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:200
no of non_attended P300 events:1000
no of labled P300 events:180
no of non_attended P300 events:900
(1080, 32, 192)
(1080,)
(3480, 32, 192)
(3480,)
(3480, 4096)
(3480,)
(3480, 32, 192)
(3480, 2)
[0. 0. 0. ... 0. 0. 0.]
[0 1 0 ... 0 0 0]
RG+xDawn Classification accuracy: 0.936494 
RG+xDawn Overall Categorical Accuracy: 93.65%
              Precision    Recall   F-Score  Support
Non-Attended   0.954222  0.970345  0.962216   2900.0
Attended       0.838041  0.767241  0.801080    580.0
